### Helper Methods for getting data

- The functions are described in their corresponding docstring 
- Each serving a different purpose to get or extract different types of data

In [77]:
from time import sleep
import pandas as pd 
import numpy as np
import jikanpy as jikan
import json as json

jikan = jikan.Jikan()

def get_anime_data(anime_id):
    anime = jikan.anime(anime_id)
    df = pd.DataFrame.from_dict(anime)
    return df

def get_top_manga():
    top_manga = jikan.top(type="manga")
    return top_manga

def get_top_anime():
    """
    Gets the top current anime from the Jikan API / MAL Database

    Returns:
    top_anime: Dict containting the top anime from the first page (25 entries)
    """    ''''''
    top_anime = jikan.top(type="anime")
    return top_anime

def extract_top_anime_names(top_anime):
    '''
    Extracts the names of the top anime from the top anime data
    
    Parameters:
    top_anime: dict
    
    Returns:
    df: DataFrame with anime names
    '''
    return pd.DataFrame(top_anime["data"])["title_english"]

def get_review_texts(anime_id, num_pages=5):
    '''
    Gets the review texts for a given anime id
    
    Parameters:
    anime_id: int
    num_pages: int, default 5 pages
    
    Returns:
    df: DataFrame with review texts
    '''
    df = pd.DataFrame()
    for i in range(1, num_pages):
        review = jikan.anime(anime_id, extension="reviews", page=i)
        if(review["data"] == []):
            break  # data equals empty list if there are no more reviews
        else:
            sleep(2)  # to prevent rate limiting
        new_data = pd.DataFrame.from_dict(review["data"])["review"]
        df = pd.concat([df, new_data], ignore_index=True)
    return df


In [7]:
anime = get_top_anime()
manga = get_top_manga()
df =  pd.DataFrame(anime["data"])["title_english"]
df = pd.DataFrame(df[0:10], columns=["title_english"])
df["Scores"] = pd.DataFrame(anime["data"])["score"]
print("Top 10 Anime")
print(df)

count_Gintama = pd.DataFrame(anime["data"])["title_english"].str.contains("Gintama").sum()
count_top_anime = pd.DataFrame(anime["data"])["title_english"].count()
print("Amount of Gintama in the top anime list: " + str(count_Gintama) + " with a total of " + str(count_top_anime) + " anime in the list.")



Top 10 Anime
                      title_english  Scores
0     Frieren: Beyond Journey's End    9.36
1  Fullmetal Alchemist: Brotherhood    9.09
2                       Steins;Gate    9.07
3                  Gintama Season 4    9.06
4   Attack on Titan Season 3 Part 2    9.05
5           Gintama: The Very Final    9.04
6                  Gintama Season 2    9.03
7                   Hunter x Hunter    9.03
8                Gintama: Enchousen    9.02
9     Legend of the Galactic Heroes    9.02
Amount of Gintama in the top anime list: 8 with a total of 25 anime in the list.


In [64]:
seasons = jikan.seasons(extension="now")
seasons

dataframe = pd.DataFrame(seasons["data"])
dataframe["title_english"]

0                                             One Piece
1     KonoSuba: God's Blessing on This Wonderful Wor...
2     Demon Slayer: Kimetsu no Yaiba Hashira Trainin...
3                                           Case Closed
4     Mushoku Tensei: Jobless Reincarnation Season 2...
5                                           Kaiju No. 8
6      That Time I Got Reincarnated as a Slime Season 3
7                             My Hero Academia Season 7
8                  Tsukimichi: Moonlit Fantasy Season 2
9                                                  None
10         Spice and Wolf: Merchant Meets the Wise Wolf
11           The Misfit of Demon King Academy II Part 2
12    An Archdemon's Dilemma: How to Love Your Elf B...
13    I Was Reincarnated as the 7th Prince so I Can ...
14                                Go! Go! Loser Ranger!
15    Chillin' in Another World with Level 2 Super C...
16          The Irregular at Magic High School Season 3
17                                        Date A

In [47]:
df = get_review_texts(5114)
df
# mal_ids = df["data"][0]["mal_id"]
# mal_ids

0     First of all, I have seen the original FMA and...
1     Fullmetal Alchemist: Brotherhood gets an immen...
2     Adaptations have long been a thorn in the side...
3     Since I couldn't find any legitimate objective...
4     *Spoiler Warning Aside from the usual everyday...
5     I can't even watch the original FMA anime now ...
6     Now, this is a prime example of how to adapt a...
7     "We're not gods, we're only human. We can't ev...
8     If one wishes to obtain something, something o...
9     Full metal alchemist brotherhood the anime tha...
10    One of my favorite anime of all time! I was on...
11    I have followed both the anime and manga of FM...
12    Ah, Fullmetal Alchemist, what an amazing ride ...
13    I was dreading having to make my way through B...
14    I'm going to be completely honest here... this...
15    There isn't much more I could say with all the...
16    This anime is going in the All Time Favourites...
17    Story: Well this story is nothing short of

In [68]:
from time import sleep


# review = jikan.anime(5114, extension="reviews", page=500)
# if(review["data"] == []):
#     print("No reviews")

df = pd.DataFrame()

for i in range(1, 3):
    review = jikan.anime(5114, extension="reviews", page=i)
    if(review["data"] == []):
        break 
    else:
        sleep(1)
    new_data = pd.DataFrame.from_dict(review["data"])["review"]
    df = pd.concat([df, new_data], ignore_index=True)

df

,0
0,"First of all, I have seen the original FMA and..."
1,Fullmetal Alchemist: Brotherhood gets an immen...
2,Adaptations have long been a thorn in the side...
3,Since I couldn't find any legitimate objective...
4,*Spoiler Warning Aside from the usual everyday...
5,I can't even watch the original FMA anime now ...
6,"Now, this is a prime example of how to adapt a..."
7,"""We're not gods, we're only human. We can't ev..."
8,"If one wishes to obtain something, something o..."
9,Full metal alchemist brotherhood the anime tha...


In [76]:
df = get_review_texts(5114, 3)
df

,0
0,"First of all, I have seen the original FMA and..."
1,Fullmetal Alchemist: Brotherhood gets an immen...
2,Adaptations have long been a thorn in the side...
3,Since I couldn't find any legitimate objective...
4,*Spoiler Warning Aside from the usual everyday...
5,I can't even watch the original FMA anime now ...
6,"Now, this is a prime example of how to adapt a..."
7,"""We're not gods, we're only human. We can't ev..."
8,"If one wishes to obtain something, something o..."
9,Full metal alchemist brotherhood the anime tha...


In [78]:
top_anime = get_top_anime()
top_anime_names = extract_top_anime_names(top_anime)
top_anime_names

0                         Frieren: Beyond Journey's End
1                      Fullmetal Alchemist: Brotherhood
2                                           Steins;Gate
3                                      Gintama Season 4
4                       Attack on Titan Season 3 Part 2
5                               Gintama: The Very Final
6                                      Gintama Season 2
7                                       Hunter x Hunter
8                                    Gintama: Enchousen
9                         Legend of the Galactic Heroes
10            Kaguya-sama: Love is War - Ultra Romantic
11                      Bleach: Thousand-Year Blood War
12                                     Gintama Season 5
13                      Fruits Basket: The Final Season
14                                              Gintama
15                                 Clannad: After Story
16                                       A Silent Voice
17                March Comes In Like a Lion 2nd